In [2]:
import psycopg2
import pandas as pd
from datetime import datetime, timedelta

exposures = psycopg2.connect('service=exposure')
start_date = (datetime.utcnow() - timedelta(days=30))

def plot_multi(data, cols=None, spacing=.1, **kwargs):

    from pandas import plotting

    # Get default color style from pandas - can be changed to any other color list
    if cols is None: cols = data.columns
    if len(cols) == 0: return
    colors = getattr(getattr(plotting, '_matplotlib').style, '_get_standard_colors')(num_colors=len(cols))

    # First axis
    ax = data.loc[:, cols[0]].plot(label=cols[0], color=colors[0], **kwargs)
    ax.set_ylabel(ylabel=cols[0])
    lines, labels = ax.get_legend_handles_labels()

    for n in range(1, len(cols)):
        # Multiple y-axes
        ax_new = ax.twinx()
        ax_new.spines['right'].set_position(('axes', 1 + spacing * (n - 1)))
        data.loc[:, cols[n]].plot(ax=ax_new, label=cols[n], color=colors[n % len(colors)], **kwargs)
        ax_new.set_ylabel(ylabel=cols[n])

        # Proper legend position
        line, label = ax_new.get_legend_handles_labels()
        lines += line
        labels += label

    ax.legend(lines, labels, loc=0)
    return ax

In [14]:
incidents = pd.read_sql_query(f'SELECT _id, name, COUNT(*) AS dept_users_count FROM "Departments" JOIN "UserDepartments" as ud ON ud.department_id = _id GROUP BY 1', exposures)
incidents = incidents.loc[incidents['dept_users_count'] != 0]
incidents

,_id,name,dept_users_count
0,89530,Munhall Volunteer Fire Company 5,1
1,76680,Carol Stream Fire Protection District,1
2,85397,Joplin Fire Department,2
3,85848,Klein Volunteer Fire Department,1
4,84631,Huber Heights Fire Department,12
...,...,...,...
2603,78596,Concordia Parish Fire District # 2,1
2604,96223,Stone Harbor Volunteer Fire Company #1,1
2605,76926,Cedar Point Fire Department,1
2606,200000,Abbotsford Fire Rescue,6
